# Allocation Algorithm

In [1]:
import pandas as pd

## Data Importing

In [253]:
#Import file
file_path = '2024-02-20_Case_Granular_Energy.xlsx'

In [344]:
# Load consumption & generation data from different portfolios and format it 
abc_demand = pd.read_excel(file_path,
    sheet_name='ABC',
    header=0,  
    usecols="A:C", 
    parse_dates=['Period']
    ).fillna(0).round()
abc_demand['Portfolio'] = 'ABC'


xyz_demand = pd.read_excel(file_path,
    sheet_name='XYZ',
    header=0,  
    usecols="A:C", 
    parse_dates=['Period']
    ).fillna(0).round()
xyz_demand['Portfolio'] = 'XYZ'

super_green_demand = pd.read_excel(file_path,
    sheet_name='Super Green',
    header=0,
    usecols="A:C",
    parse_dates=['Period']  
    ).fillna(0).round()
super_green_demand['Portfolio'] = 'Super Green'


generation_data = pd.read_excel(file_path,  
    sheet_name='Generation',
    header=0,
    usecols="A:D",
    parse_dates=['Generation month']  
    ).fillna(0).round()

abc_demand.columns



Index(['Consumer Name', 'Period', 'Consumption', 'Portfolio'], dtype='object')

In [216]:
#Concatenate the entire demand into one data frame
combined_demand = pd.concat([abc_demand, xyz_demand, super_green_demand], ignore_index=True)  
combined_demand.shape

(1826, 4)

In [260]:
#Make sure the columns are in the right formate 
combined_demand['Consumption'] = pd.to_numeric(combined_demand['Consumption'], errors='coerce').fillna(0)
generation_data['Quantity'] = pd.to_numeric(generation_data['Quantity'], errors='coerce').fillna(0)


,Generation month,plant,Technology,Quantity
0,2022-04-01,plant 207,Photovoltaic,2000
1,2022-04-01,plant 247,Hydro,17000
2,2022-04-01,plant 21,Photovoltaic,18000
3,2022-04-01,plant 276,Wind,50000
4,2022-04-01,plant 108,Photovoltaic,6000
...,...,...,...,...
851,2023-03-01,plant 253,Off-shore wind,81000
852,2023-03-01,plant 219,Off-shore wind,5000
853,2023-03-01,plant 93,Photovoltaic,21000
854,2023-03-01,plant 141,Photovoltaic,3000


## Perform Allocation Algorithm

In [353]:
def refine_allocation(combined_demand, generation_data):
    allocations = []
    allocation_id = 1  
    unmatched_capacities = []

    tech_preferences = {
        'XYZ': ['Solar', 'Wind'],
        'Super Green': ['Wind', 'Solar', 'Hydro'],
        'ABC': ['Wind', 'Solar', 'Hydro', 'Biogas', 'Biomass', 'Landfill Gas', 'Off-shore wind', 'Photovoltaic', 'Sewage gas']
    }

    combined_demand['Consumption'] = combined_demand['Consumption'].clip(lower=0)

    for portfolio, preferences in tech_preferences.items():
        demand_df = combined_demand[combined_demand['Portfolio'] == portfolio]

        for month, demand_group in demand_df.groupby('Period'):
            for tech in preferences:
                tech_data = generation_data[(generation_data['Technology'] == tech) & (generation_data['Generation month'] == month)]
                available_quantity = tech_data['Quantity'].sum()

                for index, row in demand_group.iterrows():
                    if available_quantity <= 0:
                        break  # Exit if no available generation left

                    consumer_demand = row['Consumption']
                    consumer_name = row['Consumer Name'] 

                    allocated_quantity = min(available_quantity, consumer_demand)
                    allocations.append({
                        'Allocation ID': allocation_id,
                        'Month': month.strftime('%Y-%m'),
                        'Portfolio': portfolio,
                        'Consumer': consumer_name,
                        'Technology': tech,
                        'Allocated Quantity': allocated_quantity
                    })
                    allocation_id += 1
                    # Reduce available generation by allocated amount
                    available_quantity -= allocated_quantity 

                    if consumer_demand > allocated_quantity:
                        # Record unmatched demand for this consumer
                        unmatched_capacities.append({
                            'Month': month.strftime('%Y-%m'),
                            'Portfolio': portfolio,
                            'Unmatched Quantity': consumer_demand - allocated_quantity
                        })

    allocations_df = pd.DataFrame(allocations).set_index('Allocation ID')
    unmatched_capacities_df = pd.DataFrame(unmatched_capacities).sort_values(by=['Month', 'Portfolio'])

    return allocations_df, unmatched_capacities_df



In [355]:
allocation, unmatched_capacity = refine_allocation(combined_demand, generation_data)
unmatched_capacity.head(20)

,Month,Portfolio,Unmatched Quantity
26,2022-04,ABC,33017.6
27,2022-04,ABC,7974.7
28,2022-04,ABC,5256.5
3,2022-04,Super Green,92154.0
4,2022-04,Super Green,420154.0
29,2022-05,ABC,63439.2
30,2022-05,ABC,201.0
31,2022-05,ABC,46439.2
32,2022-05,ABC,1439.2
33,2022-05,ABC,80439.2


In [296]:
!pip install openpyxl
!pip install --upgrade pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 9.3 MB/s eta 0:00:00m eta 0:00:010:0101
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandasai 1.5.18 requires pandas==1.5.3, but you have pandas 2.2.0 which is incompatible.
ydata-profiling 4.1.2 requires matplotlib<3.7,>=3.2, but you have matplotlib 3.8.2 which is incompatible.
ydata-profiling 4.1.2 requires pandas!=1.4.0,<1.6,>1.1, but you have pandas 2.2.0 which is incompatible.


## Write Allocation Results to Excel

In [357]:
# Add allocation table and unmatched capacities to excel file 
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
    # Write the DataFrame to a new sheet named 'Allocations'
    allocation.to_excel(writer, sheet_name='Allocations', index=False)
    unmatched_capacity.to_excel(writer, sheet_name='Unmatched Quantity', index=False)

## Final Check 

In [358]:
result_check = pd.read_excel(file_path,
    sheet_name='Allocations').groupby('Portfolio')
result_check.head()

,Month,Portfolio,Consumer,Plant,Quantity
0,2022-04,ABC,Customer11,Wind,124158.9
1,2022-04,ABC,Customer11,Hydro,78000.0
2,2022-04,ABC,Customer11,Biogas,8000.0
3,2022-04,ABC,Customer11,Biomass,21000.0
4,2022-04,ABC,Customer11,Landfill Gas,124158.9
7,2022-04,Super Green,Customer21,Wind,406000.0
8,2022-04,Super Green,Customer21,Hydro,78000.0
9,2022-04,Super Green,Customer22,Wind,-406000.0
10,2022-04,Super Green,Customer22,Hydro,-78000.0
11,2022-04,Super Green,Customer23,Wind,406000.0
